<a href="https://colab.research.google.com/github/Explorer9/INFO5731_FALL2020/blob/master/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [30]:
# Write your code here
import pandas as pd
import requests
import matplotlib.pyplot as plt
import collections
import spacy
from bs4 import BeautifulSoup
def insert_into_array(values,ref_array):
  for i in values:
    ref_array.append(i.text)
URL=["https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv","https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbeqe2tiyac72wh5mjwrhq4mbzhzfmxvlnomwklyczuf43o6ss6oe3ftnbndj4k4riocrdyzu6sgyppvhkaystluja","https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbpry3tozal7gthtnbtrhq42bj6y4hhzo5ziwr26fbyhvrl4ty4oyyvjnjjcvundtwut6qkcyjjczgdiuulalo5ckxc","https://www.imdb.com/title/tt7286456/reviews/_ajax?ref_=undefined&paginationKey=g4wp7cbnq43da3yf7cvhxnbsrlu42abhzfmxvlnomwklyczuf43o6ss6oe2fxnbndb4k4hreyfcjga437josyy4nw6m77si"]
text=[]
rating=[]
review_title=[]
for i in URL:
  page=requests.get(i,headers={'User-Agent':'Chrome/85.0.4183.121'})
  soup = BeautifulSoup(page.content, 'html.parser')
  insert_into_array(soup.find_all('div', class_='text show-more__control'),text)
  insert_into_array(soup.find_all('a',class_='title'),review_title)
  insert_into_array(soup.find_all(name='span',class_='rating-other-user-rating'),rating)
imdb_df=pd.DataFrame({'comment':text,'review_title':review_title})
imdb_df.to_csv('imdb.csv')



In [4]:
(1)
import re
from nltk.util import ngrams
def generate_ngrams(s, n1):
  s = s.lower()
  s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
  tokens = [token for token in s.split(" ") if token != ""]
  output = list(ngrams(tokens, n1))
  return output
def ngram_freq(inp,n2):
  d=[]
  for i in inp:
    for j in generate_ngrams(i, n2):
      d.append(j)
  counter=collections.Counter(d)
  return counter
three_gram=ngram_freq(imdb_df['comment'],3)

In [ ]:
(2)
one=ngram_freq(imdb_df['comment'],1)
two=ngram_freq(imdb_df['comment'],2)
prob={}
for i in two.keys():
  prob[i]=two[i]/one[eval("'"+i[0]+"'"+",")]

In [41]:
prob

{('2019', 'one'): 0.6666666666666666,
 ('a', 'better'): 0.0045662100456621,
 ('a', 'comic'): 0.0091324200913242,
 ('a', 'lot'): 0.02968036529680365,
 ('a', 'masterpiece'): 0.02968036529680365,
 ('a', 'movie'): 0.04337899543378995,
 ('a', 'proper'): 0.0091324200913242,
 ('a', 'typical'): 0.00684931506849315,
 ('a', 'while'): 0.00228310502283105,
 ('action', 'riddled'): 0.1,
 ('all', 'it'): 0.03571428571428571,
 ('alone', 'and'): 0.25,
 ('an', 'eye'): 0.017857142857142856,
 ('an', 'impact'): 0.017857142857142856,
 ('and', 'cringy'): 0.002380952380952381,
 ('and', 'every'): 0.004761904761904762,
 ('and', 'isolated'): 0.002380952380952381,
 ('and', 'make'): 0.007142857142857143,
 ('and', 'realistically'): 0.004761904761904762,
 ('and', 'scenography'): 0.002380952380952381,
 ('and', 'think'): 0.004761904761904762,
 ('and', 'truly'): 0.009523809523809525,
 ('and', 'you'): 0.014285714285714285,
 ('at', 'the'): 0.25862068965517243,
 ('at', 'times'): 0.08620689655172414,
 ('become', 'a'): 0.333

In [8]:
noun_phrases=[]
nlp = spacy.load("en_core_web_sm")
for i in imdb_df['comment']:
  doc2 = nlp(i)
  for chunk in doc2.noun_chunks:
    noun_phrases.append(chunk)
noun_freq=collections.Counter(noun_phrases)

In [39]:
noun_phrases[1:15]

[a movie,
 an impact,
 Joaquin's performance,
 scenography,
 it,
 brilliance,
 Grotesque,
 haunting,
 cringy,
 times,
 you,
 an eye,
 it,
 seriously funny moments]

In [9]:
(3)maxi=max(noun_freq.values())
relative_freq=[]
for i in noun_freq.values():
  relative_freq.append(i/maxi)
d={'noun_phrases':noun_phrases,'relative_freq':relative_freq}
relative_freq_df=pd.DataFrame(data=d)
relative_freq_df

,noun_phrases,relative_freq
0,"(a, while)",1.0
1,"(a, movie)",1.0
2,"(an, impact)",1.0
3,"(Joaquin, 's, performance)",1.0
4,(scenography),1.0
...,...,...
3537,"(the, story)",1.0
3538,(It),1.0
3539,"(a, dark, movie)",1.0
3540,"(the, problem)",1.0


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [27]:
# (1)Write your code here
def TermDocumentMatrix(docs, docIDs=None):
    vectorizer = CountVectorizer(lowercase=True, stop_words=None)
    tdm = vectorizer.fit_transform(docs)
    tdm_feature_names = vectorizer.get_feature_names()
    #
    df = pd.DataFrame(tdm.toarray(), columns=tdm_feature_names, dtype="float64")
    if docIDs is not None:
        df.index = docIDs    
    return df
TermDocumentMatrix(imdb_df['comment'])



,10,100,10grade,12y,20,2019,30mins,30s,37,40,45,52,60s,65,70,70s,80s,90,aaahs,able,about,above,absolute,absolutely,absurd,academy,accepted,acclaim,accolades,accomplishes,account,accurate,achievement,achivement,act,acted,acting,action,actions,activities,...,wonderful,word,words,work,worked,workers,world,worse,worst,worth,worthy,would,wouldn,wow,write,writer,writers,writing,written,wrong,wrote,wtf,yas,yawn,yeah,year,years,yes,yesterday,yet,you,young,younger,your,yours,yourself,yourselves,youtube,zazie,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
(2)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

# Download stopwords list
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

# Interface lemma tokenizer from nltk with sklearn
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]

# Lemmatize the stop words
tokenizer=LemmaTokenizer()
token_stop = tokenizer(' '.join(stop_words))

search_terms = 'movie'
documents = imdb_df['comment']

# Create TF-idf model
vectorizer = TfidfVectorizer(stop_words=token_stop, 
                              tokenizer=tokenizer)
doc_vectors = vectorizer.fit_transform([search_terms] + documents)

# Calculate similarity
cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
document_scores = [item.item() for item in cosine_similarities[1:]]
# [0.0, 0.287]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
cosine_similarities

array([1.        , 0.05903803, 0.05808453, 0.05951325, 0.07513543,
       0.05134695, 0.04116796, 0.06211001, 0.03158162, 0.00974154,
       0.07003319, 0.02988736, 0.02919585, 0.01855501, 0.04782592,
       0.06121177, 0.05759602, 0.        , 0.05348604, 0.07149172,
       0.02183707, 0.00366208, 0.0595138 , 0.07772352, 0.03051821,
       0.0510454 , 0.03032058, 0.01012196, 0.01587755, 0.08755906,
       0.05310819, 0.04839483, 0.00593308, 0.03441265, 0.09335265,
       0.        , 0.07005507, 0.01249122, 0.09778998, 0.09318248,
       0.03704431, 0.01450643, 0.03933001, 0.05192846, 0.0811986 ,
       0.02834566, 0.07364021, 0.0164924 , 0.04015071, 0.        ,
       0.02834566, 0.07364021, 0.0164924 , 0.04015071, 0.        ,
       0.01039455, 0.        , 0.        , 0.07529993, 0.03943023,
       0.00868184, 0.        , 0.04513494, 0.02443399, 0.01715675,
       0.06958998, 0.08888881, 0.04139648, 0.01362704, 0.05948853,
       0.09585775, 0.04644629, 0.05949413, 0.03655955, 0.05807

# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# The GitHub link of your final csv file

 Link: https://github.com/Explorer9/prabhu_INFO5731/blob/master/Jokersentiments.csv